### Task 103: Integrate interest APS calculation from CPT liquidity pool in Rebasing token

* Make the the APS source configurable. The CPT pool being used to calculate the interest APS should be configurable for any CPT in the simulation.

* Being able to process only one source is sufficient. End goal is to be able to proportionally aggregate the interest APS from several sources. For example, the TCT may rebase using the supplies of T1 and T2 managed by the vault, including the interest rates for liquidity containing those assets. This may be a single pool that contains both tokens, and/or pools that contain only one of the assets.


In [1]:
import os
import copy
import numpy as np
import time
import math
import matplotlib.pyplot as plt
cwd =  os.getcwd().replace("notebooks/test","")
os.chdir(cwd)

from python.dev.event import Deposit
from python.dev.event import Withdraw
from python.dev.cpt import Liquidity
from python.dev.token import RebaseToken
from python.dev.math.model import TimeDeltaModel
from python.dev.math.model import TokenDeltaModel
from python.dev.math.interest.ips import ConstantIPS
from python.dev.math.interest.ips.aggregate import AggregateIPS
from python.dev.math.interest import CompoundReturn

In [2]:
time0 = time.time()
N_TOKENS = 2

### Part A: Test IPS

In [3]:
APY = 0.1234567
TYPE_DEPOSIT_TOKEN = 'DEPOSIT'
A0 = 10000
token_delta = 1000
time_delta = 1000

token = RebaseToken(TYPE_DEPOSIT_TOKEN)
addr = token.gen_address()
token.init_token(time0, addr)
token.add_event(Deposit(APY, A0, 0, addr))

token.add_event(Deposit(APY, token_delta, time_delta, addr))
token.add_event(Deposit(APY, token_delta, time_delta, addr))
token.add_event(Deposit(APY, token_delta, time_delta, addr))

last_state = token.get_state_series(addr).get_last_state()

cIPS = ConstantIPS()
ips = cIPS.calc_ips_from_state(last_state)

In [4]:
cReturn = CompoundReturn()
r = cReturn.apply(A0,ConstantIPS.SECONDS_YEAR,ips)
print('Test 1: annual return {} -> pass {}'.format(r, r == A0*(APY+1)))

Test 1: annual return 11234.567 -> pass True


### Part B: Liquidity Pool Simulation Run
* Initialize Rebasing/Rebasing Tokens

In [5]:
APY = 0.1
TYPE_DEPOSIT_TOKEN = 'DEPOSIT'
TOKEN = N_TOKENS * [None]
ADDR = N_TOKENS * [None]
init_deposit = 10000 

TOKEN[0] = RebaseToken(TYPE_DEPOSIT_TOKEN)
ADDR[0] = TOKEN[0].gen_address()
TOKEN[0].init_token(time0, ADDR[0])
TOKEN[0].add_event(Deposit(APY, init_deposit, 0, ADDR[0]))
   
TOKEN[1] = RebaseToken(TYPE_DEPOSIT_TOKEN)
ADDR[1] = TOKEN[1].gen_address()
TOKEN[1].init_token(time0, ADDR[1])
TOKEN[1].add_event(Deposit(APY, init_deposit, 0, ADDR[1]))
    
x = TOKEN[0].get_balance_deposits()
y = TOKEN[1].get_balance_deposits()

# Initialize Liquidity Pool
liq = Liquidity(x,y)

* simulate liquidity pool

In [6]:
percent = 0.05
N = 250
prices_yx = N * [0]
prices_xy = N * [0]

for k in range(N):

    token_delta = TokenDeltaModel(1,100).apply() 
    time_delta = TimeDeltaModel().apply() 
    TOKEN[0].add_event(Deposit(APY, token_delta, time_delta, ADDR[0]))

    token_delta = TokenDeltaModel(1,100).apply() 
    time_delta = TimeDeltaModel().apply() 
    TOKEN[1].add_event(Deposit(APY, token_delta, time_delta, ADDR[1]))        
        
    T1_yield = TOKEN[0].get_state_series(ADDR[0]).get_last_state().get_yield()
    T2_yield = TOKEN[1].get_state_series(ADDR[1]).get_last_state().get_yield()
    
    T1_delta = TOKEN[0].get_state_series(ADDR[0]).get_last_state().get_delta()
    T2_delta = TOKEN[1].get_state_series(ADDR[1]).get_last_state().get_delta()
        
    x = TOKEN[0].get_balance_deposits()
    y = TOKEN[1].get_balance_deposits()

    liq.delta_x(T1_delta+T1_yield)
    liq.delta_y(T2_delta+T2_yield)     
    L = liq.calc()
    prices_yx[k] = liq.get_swap_price(Liquidity.YX_PRICE)  
    prices_xy[k] = liq.get_swap_price(Liquidity.XY_PRICE)
    
    if (k % int(N/(100*percent)) == 0):
        out_str = '[SIM {}]'.format(k)
        for k in range(N_TOKENS):
            amt = TOKEN[k].get_balance_deposits()
            out_str = out_str+' T{}: {:.2f}'.format(k+1,amt)
            
        print(out_str) 
        

[SIM 0] T1: 10117.86 T2: 10060.50
[SIM 50] T1: 13600.97 T2: 15225.07
[SIM 100] T1: 19401.60 T2: 20503.42
[SIM 150] T1: 24891.90 T2: 25358.91
[SIM 200] T1: 30813.37 T2: 31651.21


* aggregate interest per second (ips) from each token

In [7]:
T1_last_state = TOKEN[0].get_state_series(ADDR[0]).get_last_state()
T2_last_state = TOKEN[1].get_state_series(ADDR[1]).get_last_state()

aggr = AggregateIPS(ConstantIPS())
ipsAggr = aggr.apply([T1_last_state, T2_last_state])

cReturn = CompoundReturn()
returnAggr = cReturn.apply(init_deposit, ConstantIPS.SECONDS_YEAR, ipsAggr)

print('The balance after investing {} into this pool over one year is {}'.format(init_deposit, returnAggr))


The balance after investing 10000 into this pool over one year is 11000.0
